<a href="https://colab.research.google.com/github/adrianortega93/Paradigmas-de-Programacion/blob/main/Tarea1_Desafio_Ortega_Adrian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
    <h2>Universidad Casa Grande</h2>
    <h3>Maestría en Inteligencia Artificial y Ciencia de Datos</h3>
    <p><strong>Autor:</strong> Adrián Ortega Q.</p>
</div>
<div style="text-align: center;">
    <h3>Tarea 1 (Desafío)</h3>
</div>

In [ ]:
# =======================================================================================
# Tarea 1 (Desafío): Construcción de un Pipeline de Clasificación y Análisis de Modelos
# Curso: Paradigmas de Programación para IA y Ciencia de Datos
# Creado por: Adrián Ortega Q.
# =======================================================================================

# ---------------------------------------------------------------------------------------
# Análisis Exploratorio y Preparación de Datos (Como se vio en la práctica)
# ---------------------------------------------------------------------------------------
print("\n--- Análisis Exploratorio y Preparación de Datos (Como se vio en la práctica) ---")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Componentes clave de Scikit-learn para el desafío
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Tarea 1 (Desafío): Librerías cargadas.")


In [ ]:
# ---------------------------------------------------------------------------------------
# Identificación y División de Datos
# ---------------------------------------------------------------------------------------

# Para esta tarea cargaremos los datos desde un archivo ai4i2020.csv,
dataFile = 'ai4i2020.csv'

try:
  print("\n--- Identificación y División de Datos ---")
  print("Tarea 1 (Desafío): DataFrame")
  # Cargar los datos desde el archivo ai4i2020.csv
  dfFrame = pd.read_csv(dataFile)
  #print(dfFrame)

  # a) Identificar características y objetivo
  camposDescartados = ['UDI', 'Product ID']
  X = dfFrame.drop(camposDescartados, axis=1)
  y = dfFrame['Machine failure']
  #print(X)

  # b) Separar columnas por tipo
  numeric_features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'TWF',  'HDF',  'PWF',  'OSF',  'RNF' ]
  categorical_features = ['Type']

  # c) División en entrenamiento y prueba (¡fundamental para una evaluación honesta!)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
  print(f"\nDatos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")


except FileNotFoundError:
    print(f"Error: El archivo '{dfFrame}' no se encontró. Asegúrate de que la ruta sea correcta.")
except Exception as e:
    print(f"Ocurrió un error: {e}")


In [ ]:
# ---------------------------------------------------------------------------------------
# Pipeline de Preprocesamiento Automatizado (Aplicando POO y el Paradigma Descriptivo)
# ---------------------------------------------------------------------------------------
print("\n--- Pipeline de Preprocesamiento Automatizado (Aplicando POO y el Paradigma Descriptivo) ---")

# ---------------------------- Paradigma Orientado a Objetos ----------------------------
# Enfoque: Encapsulamos la lógica de pipelines y como mezclarlos
# La función preprocessor devolverá ColumnTransformer

class ProcesamientoModel:

  def __init__(self, numeric_features: list, categorical_features: list):
    """
     Constructor para ProcesamientoModel.
     Args:
      numeric_features (list): Lista numérica.
      categorical_features (list): Lista categórica.
    """
    self.numeric_features = numeric_features
    self.categorical_features = categorical_features

  def get_preprocess(self):
    """
     Crea y retorna un ColumnTransformer para el preprocesamiento de datos,
     usando las características definidas en el constructor.
    """
    # Aquí se aplica el enfoque Descriptivo
    # ya que se declara lo que se necesita de Pipeline
    numeric_pipeline = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_pipeline, self.numeric_features),
            ('cat', categorical_pipeline, self.categorical_features)
        ],
        remainder='passthrough' # Mantiene columnas no especificadas (si las hubiera)
    )
    print("Preprocesador (ColumnTransformer) creado exitosamente.")
    return preprocessor

# --- Demostración de uso ---
# Se instancia la clase y se llama a la función
numeric_selector = ProcesamientoModel(numeric_features, categorical_features)
numeric_selector.get_preprocess()



In [ ]:
# ---------------------------------------------------------------------------------------
# Comparación de Modelos dentro de un Pipeline Completo
# ---------------------------------------------------------------------------------------

print("\n--- Comparación de Modelos dentro de un Pipeline Completo ---")

#Asignamos a una variable la funcion que se instanció de la clase ProcesamientoModel
preprocessor = numeric_selector.get_preprocess()

# a) Pipeline con Regresión Logística
pipeline_logreg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# b) Pipeline con Random Forest
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# c) Entrenar y evaluar cada modelo para su commparación
print("\n==*Entrenando modelo de Regresión Logística*==")
pipeline_logreg.fit(X_train, y_train)

print("\n=== Evaluación de Regresión Logística ===")
predictions_logreg = pipeline_logreg.predict(X_test)
print(classification_report(y_test, predictions_logreg))

print("==*Entrenando modelo de Random Forest*==")
pipeline_rf.fit(X_train, y_train)

print("\n=== Evaluación de Random Forest ===")
predictions_rf = pipeline_rf.predict(X_test)
print(classification_report(y_test, predictions_rf))


In [ ]:
# ---------------------------------------------------------------------------------------
#  Análisis Profundo del Mejor Modelo
# ---------------------------------------------------------------------------------------

# Obtener los reportes en forma de diccionario
report_logreg = classification_report(y_test, predictions_logreg, output_dict=True)
report_rf = classification_report(y_test, predictions_rf, output_dict=True)

# Crear la tabla comparativa con las métricas promedio
comparacion = pd.DataFrame({
    'Precisión': [
        report_logreg['weighted avg']['precision'],
        report_rf['weighted avg']['precision']
    ],
    'Recall': [
        report_logreg['weighted avg']['recall'],
        report_rf['weighted avg']['recall']
    ],
    'F1-score': [
        report_logreg['weighted avg']['f1-score'],
        report_rf['weighted avg']['f1-score']
    ],
    'Accuracy': [
        accuracy_score(y_test, predictions_logreg),
        accuracy_score(y_test, predictions_rf)
    ]
}, index=['Regresión Logística', 'Random Forest'])

# Redondear para mejor visualización
comparacion = comparacion.round(4)

print("\n=== Comparación de modelos ===")
print(comparacion)


**Selección y Justificación:**

Aunque ambos modelos muestran un rendimiento excepcional, el Random Forest es marginalmente superior a la Regresión Logística debido a su Recall perfecto (1.00) para la clase '1' (Fallo).

En el contexto de la predicción de fallos de maquinaria, la capacidad de detectar cada fallo real es de suma importancia para la toma de decisiones de negocio y para evitar interrupciones operacionales y costos asociados. Un Recall del 1.00 garantiza que ninguna máquina en riesgo de fallo pase desapercibida por el modelo. Dado que la Precisión y Accuracy son igualmente perfectas para ambos, el Random Forest ofrece la máxima seguridad en la detección de fallos.



In [ ]:
# ------------------------------------------------------------------------------
# 5. Análisis de Importancia de Características:
# ------------------------------------------------------------------------------
print("\n--- 5. ANÁLISIS DEL MEJOR MODELO EN ESTE CASO RANDOM FOREST---")

# a) Extraer la importancia de las características
# El pipeline nos ayuda a acceder a los pasos internos.
modelo_final_rf = pipeline_rf.named_steps['classifier']
feature_importances = modelo_final_rf.feature_importances_

# b) Obtener los nombres de las características DESPUÉS del preprocesamiento
# El OneHotEncoder crea nuevos nombres de columnas.
feature_names_raw = pipeline_rf.named_steps['preprocessor'].get_feature_names_out()

# c) Crear un DataFrame para una fácil visualización
importances_df = pd.DataFrame({
    'feature': feature_names_raw,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

top_5_features = importances_df.head(5)
print("\nTop 5 características más importantes:")
print(top_5_features)

# d) Visualizar la importancia de las características
plt.figure(figsize=(8, 5))
sns.barplot(x='importance', y='feature', data=top_5_features)
plt.title('Top 5 Características más Importantes (Random Forest)')
plt.xlabel('Importancia')
plt.ylabel('Característica')
plt.show()


print("\n¡Desafío completado, Juro solemnemente que mis intenciones no son buenas! :D.")

**Interpretación de negocio.**

Antes de emitir este parrafo realicé una prueba omitiendo los campos 'TWF',  'HDF',  'PWF',  'OSF',  'RNF'
donde el resultado es el siguiente.

Top 5 características más importantes:

                       feature | importance
8 |  remainder__Machine failure  |  0.804776

3   |          num__Torque [Nm]  |  0.102462

2 | num__Rotational speed [rpm]  |  0.043514

4  |       num__Tool wear [min]  | 0.023067

0 |    num__Air temperature [K]   | 0.012664



Donde me percaté que los resultados esperados daban exactamente el mismo para ambos casos Regresión Logística y Ramdom Forest.

Lo que me llega a dejar en claro que esos campos se deben tener en consideracion ya que no habría diferencia por no tomarse en consideración mas variables.

Sinceramente no entiendo bien el enfoque a donde va dirigido esta métrica pero evaluando los resultados de acuerdo a las pruebas puedo decir que me enfocaría en las caracteristicas que tienen mayor importancia, para mentener el mismo nivel y de ser posible mejorarlo.

Mi humilde opinión como un gerente de planta 😶


